In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
import pickle
import re

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import f1_score, confusion_matrix, precision_score, recall_score
from torch import nn
from torch import optim
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

# Neural Network:-

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(NeuralNetwork, self).__init__()
        self.layer_1 = nn.Linear(input_dim, 32)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.1)
        self.layer_2 = nn.Linear(32, 16)
        self.layer_3 = nn.Linear(16, output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.layer_1(x)
        x = self.dropout(x)
        x = self.relu(x)
        x = self.layer_2(x)
        x = self.dropout(x)
        x = self.relu(x)
        x = self.layer_3(x)

        return x

In [ ]:
input_dim = 1
hidden_dim=16
output_dim = 2

class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(NeuralNetwork, self).__init__()
        self.layer_1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU(0.2)
        self.layer_2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.layer_1(x)
        x = self.relu(x)
        x = self.layer_2(x)

        return x

In [ ]:
def trainer(X_train, y_train):
    """For training the model"""
    learning_rate = 0.01
    num_epochs = 20
    batch_size = 32
    loss_values = []

    input_dim = 1
    output_dim = 2

    model = NeuralNetwork(input_dim, output_dim)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-3)

    train_data = TensorDataset(X_train, y_train)
    train_dataloader = DataLoader(dataset=train_data,batch_size=batch_size,shuffle=True)

    for epoch in range(num_epochs):
        for Xt,yt in train_dataloader:
            # In the starting for everyt training data our gradient should be 0
            optimizer.zero_grad()

            pred = model(Xt)  # do the prediction
            loss = loss_fn(pred, yt)  # calculate loss
            loss_values.append(loss.item())   # keep storing the loss
            loss.backward()   # do backpropagation and compute the gradients of all the model parameters
            optimizer.step()   # update the parameters
    return model

In [ ]:
def tester(X_test,y_test,model):
    """For testing the model"""
    test_data = TensorDataset(X_test,y_test)

    model.eval()
    correct = 0
    total = 0
    true_labels = []
    predicted_labels = []

    with torch.no_grad():
        for Xt,yt in test_data:
            outputs = model(Xt)

            predicted = torch.argmax(nn.functional.softmax(outputs, dim=-1)).item()

            if (predicted==yt):
                correct+=1
            total+=1
            true_labels.append(yt.item())
            predicted_labels.append(predicted)

    confusions = confusion_matrix(true_labels, predicted_labels)
    print("Confusion Matrix: ", confusions)

    accuracy = correct/total
    print("Accuracy: " + str(accuracy))

    print("F1 Score",f1_score(predicted_labels, true_labels))
    print("Precision",precision_score(predicted_labels, true_labels))
    print("Recall",recall_score(predicted_labels, true_labels))


In [ ]:
def similarity_finder(embed1, embed2,isAI):
  """Function for finding similarities between two reviews using embeddings"""
  similarities = []

  if (isAI):   # whether embed1 is AI or not
    for i in range(len(embed1)):
      ind = list(embed1.keys())[i]
      try:
        similarities.append(cosine_similarity([embed1[ind]], [embed2[ind]]))
      except:
        pass

  else:
    for i in range(len(embed1)):
      ind = list(embed1.keys())[i]
      sim = []
      try:
        for j in range(len(embed1[ind])):
            sim.append(cosine_similarity([embed1[ind][j]],[embed2[ind]]))
        similarities.append(sim)
      except:
        pass

  return similarities

In [ ]:
def predictor(ai_iclr_embed, ai_neur_embed, human_iclr_embed, human_neur_embed, gpt_iclr_embed, gpt_neur_embed):
    # Loading the files
    ai_iclr = json.load(open(ai_iclr_embed))
    ai_neur = json.load(open(ai_neur_embed))
    human_iclr = json.load(open(human_iclr_embed))
    human_neur = json.load(open(human_neur_embed))
    gpt_iclr = json.load(open(gpt_iclr_embed))
    gpt_neur = json.load(open(gpt_neur_embed))

    # Finding out the similarities
    ai_iclr_sim = similarity_finder(ai_iclr, gpt_iclr,1)
    ai_neur_sim = similarity_finder(ai_neur, gpt_neur,1)
    human_iclr_sim = similarity_finder(human_iclr, gpt_iclr,0)
    human_neur_sim = similarity_finder(human_neur, gpt_neur,0)

    ai_train = [[a[0][0]] for a in ai_iclr_sim[:608]] + [[a[0][0]] for a in ai_neur_sim[:576]]
    human_train = [[a[0][0]] for revw in human_iclr_sim[:608] for a in revw] + [[a[0][0]] for revw in human_neur_sim[:576] for a in revw]
    X_train = torch.tensor(ai_train + human_train,dtype=torch.float32)
    y_train = torch.tensor([1 for i in ai_train] + [0 for i in human_train])

    # Shuffle the train dataset
    perm_indices = torch.randperm(len(X_train))
    X_train = X_train[perm_indices]
    y_train = y_train[perm_indices]

    # Training the model
    model = trainer(X_train,y_train)

    human_test_iclr = [[a[0][0]] for revw in human_iclr_sim[608:] for a in revw]
    X_test_iclr = torch.tensor([[a[0][0]] for a in ai_iclr_sim[608:]] + human_test_iclr, dtype=torch.float32)
    y_test_iclr = torch.tensor([1 for i in ai_iclr_sim[608:]] + [0 for i in human_test_iclr])

    human_test_neur = [[a[0][0]] for revw in human_neur_sim[576:] for a in revw]
    X_test_neur = torch.tensor([[a[0][0]] for a in ai_neur_sim[576:]] + human_test_neur, dtype=torch.float32)
    y_test_neur = torch.tensor([1 for i in ai_neur_sim[576:]] + [0 for i in human_test_neur])

    print("++++++++++++++++++ ICLR ++++++++++++++++++")
    tester(X_test_iclr,y_test_iclr,model)
    print("\n++++++++++++++++++ NeurIPS ++++++++++++++++++")
    tester(X_test_neur,y_test_neur,model)

In [ ]:
def trainer_tester(ai_iclr_embed_train, ai_neur_embed_train, human_iclr_embed_train, human_neur_embed_train,ai_iclr_embed_test, ai_neur_embed_test, human_iclr_embed_test, human_neur_embed_test,gpt_iclr_embed, gpt_neur_embed):
    # Training files
    ai_iclr_train = json.load(open(ai_iclr_embed_train))
    ai_neur_train = json.load(open(ai_neur_embed_train))
    human_iclr_train = json.load(open(human_iclr_embed_train))
    human_neur_train = json.load(open(human_neur_embed_train))

    # Testing files
    ai_iclr_test = json.load(open(ai_iclr_embed_test))
    ai_neur_test = json.load(open(ai_neur_embed_test))
    human_iclr_test = json.load(open(human_iclr_embed_test))
    human_neur_test = json.load(open(human_neur_embed_test))
    gpt_iclr = json.load(open(gpt_iclr_embed))
    gpt_neur = json.load(open(gpt_neur_embed))

     # Finding out the similarities
    ai_iclr_sim = similarity_finder(ai_iclr_train, gpt_iclr,1)
    ai_neur_sim = similarity_finder(ai_neur_train, gpt_neur,1)
    human_iclr_sim = similarity_finder(human_iclr_train, gpt_iclr,0)
    human_neur_sim = similarity_finder(human_neur_train, gpt_neur,0)

    ai_iclr_sim_test = similarity_finder(ai_iclr_test, gpt_iclr,1)
    ai_neur_sim_test = similarity_finder(ai_neur_test, gpt_neur,1)
    human_iclr_sim_test = similarity_finder(human_iclr_test, gpt_iclr,1)
    human_neur_sim_test = similarity_finder(human_neur_test, gpt_neur,1)

    ai_train = [[a[0][0]] for a in ai_iclr_sim] + [[a[0][0]] for a in ai_neur_sim]
    human_train = [[a[0][0]] for revw in human_iclr_sim for a in revw] + [[a[0][0]] for revw in human_neur_sim for a in revw]
    X_train = torch.tensor(ai_train + human_train,dtype=torch.float32)
    y_train = torch.tensor([1 for i in ai_train] + [0 for i in human_train])

    # Shuffle the train dataset
    perm_indices = torch.randperm(len(X_train))
    X_train = X_train[perm_indices]
    y_train = y_train[perm_indices]

    # Training the model
    model = trainer(X_train,y_train)

    human_test_iclr = [[a[0][0]] for a in human_iclr_sim_test]
    X_test_iclr = torch.tensor([[a[0][0]] for a in ai_iclr_sim_test] + human_test_iclr, dtype=torch.float32)
    y_test_iclr = torch.tensor([1 for i in ai_iclr_sim_test] + [0 for i in human_test_iclr])

    # Shuffle the test dataset
    perm_indices = torch.randperm(len(X_test_iclr))
    X_test_iclr = X_test_iclr[perm_indices]
    y_test_iclr = y_test_iclr[perm_indices]

    human_test_neur = [[a[0][0]] for a in human_neur_sim_test]
    X_test_neur = torch.tensor([[a[0][0]] for a in ai_neur_sim_test] + human_test_neur, dtype=torch.float32)
    y_test_neur = torch.tensor([1 for i in ai_neur_sim_test] + [0 for i in human_test_neur])

    # Shuffle the test dataset
    perm_indices = torch.randperm(len(X_test_neur))
    X_test_neur = X_test_neur[perm_indices]
    y_test_neur = y_test_neur[perm_indices]

    print("++++++++++++++++++ ICLR ++++++++++++++++++")
    tester(X_test_iclr,y_test_iclr,model)
    print("\n++++++++++++++++++ NeurIPS ++++++++++++++++++")
    tester(X_test_neur,y_test_neur,model)

## Predicting Actual Reviews:-

In [ ]:
predictor(f"Embeddings/ai_iclr_embed",f"Embeddings/ai_neur_embed",f"Embeddings/human_iclr_embed",f"Embeddings/human_neur_embed",f"Embeddings/Regenerated Embeddings/gpt-4/gpt_iclr_embed",f"Embeddings/Regenerated Embeddings/gpt-4/gpt_neur_embed")

## Synonym Attack (or Adjective Attack:-)

In [ ]:
embed_path = "Embeddings/"
embed_path1 = "Embeddings/Attacked Embeddings/Paraphrasing Attack/"
trainer_tester(f"{embed_path}ai_iclr_embed",f"{embed_path}ai_neur_embed",f"{embed_path}human_iclr_embed",f"{embed_path}human_neur_embed",f"{embed_path1}ai_iclr_para_attack_embed_test",f"{embed_path1}ai_neur_para_attack_embed_test",f"{embed_path1}human_iclr_para_attack_embed_test",f"{embed_path1}human_neur_para_attack_embed_test",f"{embed_path}Regenerated Embeddings/gpt-4/gpt_iclr_embed",f"{embed_path}Regenerated Embeddings/gpt-4/gpt_neur_embed")

## Paraphrasing Attack:-

In [ ]:
embed_path = "Embeddings/"
embed_path1 = "Embeddings/Attacked Embeddings/Paraphrasing Attack/"
trainer_tester(f"{embed_path}ai_iclr_embed",f"{embed_path}ai_neur_embed",f"{embed_path}human_iclr_embed",f"{embed_path}human_neur_embed",f"{embed_path1}ai_iclr_para_attack_embed_test",f"{embed_path1}ai_neur_para_attack_embed_test",f"{embed_path1}human_iclr_para_attack_embed_test",f"{embed_path1}human_neur_para_attack_embed_test",f"{embed_path}Regenerated Embeddings/gpt-4/gpt_iclr_embed",f"{embed_path}Regenerated Embeddings/gpt-4/gpt_neur_embed")

## Modified Paraphrasing Attack:-

In [ ]:
# Training files
embed_path = "Embeddings/"
ai_iclr = json.load(open(f"{embed_path}ai_iclr_embed"))
ai_neur = json.load(open(f"{embed_path}ai_neur_embed"))
human_iclr = json.load(open(f"{embed_path}human_iclr_embed"))
human_neur = json.load(open(f"{embed_path}human_neur_embed"))

embed_path1 = "Embeddings/Attacked Embeddings/Modified Paraphrasing Attack/Modified Paraphrased/"
ai_iclr_modified_para = pickle.load(open(f"{embed_path1}ai_iclr_modified_embed_para.pkl","rb"))
ai_neur_modified_para = pickle.load(open(f"{embed_path1}ai_neur_modified_embed_para.pkl","rb"))
human_iclr_modified_para = pickle.load(open(f"{embed_path1}human_iclr_modified_embed_para.pkl","rb"))
human_neur_modified_para = pickle.load(open(f"{embed_path1}human_neur_modified_embed_para.pkl","rb"))

embed_path2 = "Embeddings/Attacked Embeddings/Modified Paraphrasing Attack/Modified Reviews/"
ai_iclr_modified = pickle.load(open(f"{embed_path2}ai_iclr_review_modified_embed.pkl","rb"))
ai_neur_modified = pickle.load(open(f"{embed_path2}ai_neur_review_modified_embed.pkl","rb"))
human_iclr_modified = pickle.load(open(f"{embed_path2}human_iclr_review_modified_embed.pkl","rb"))
human_neur_modified = pickle.load(open(f"{embed_path2}human_neur_review_modified_embed.pkl","rb"))

gpt_iclr = json.load(open(f"{embed_path}Regenerated Embeddings/gpt-4/gpt_iclr_embed"))
gpt_neur = json.load(open(f"{embed_path}Regenerated Embeddings/gpt-4/gpt_neur_embed"))

# Testing files
ai_iclr_modified_para_test = pickle.load(open(f"{embed_path1}ai_iclr_modified_para_test.pkl","rb"))
ai_neur_modified_para_test = pickle.load(open(f"{embed_path1}ai_neur_modified_para_test.pkl","rb"))
human_iclr_modified_para_test = pickle.load(open(f"{embed_path1}human_iclr_modified_para_test.pkl","rb"))
human_neur_modified_para_test = pickle.load(open(f"{embed_path1}human_neur_modified_para_test.pkl","rb"))

In [ ]:
# Finding out the similarities
ai_iclr_sim = similarity_finder(ai_iclr, gpt_iclr,1)
ai_neur_sim = similarity_finder(ai_neur, gpt_neur,1)
human_iclr_sim = similarity_finder(human_iclr, gpt_iclr,0)
human_neur_sim = similarity_finder(human_neur, gpt_neur,0)

ai_iclr_sim_modified_para = similarity_finder(ai_iclr_modified_para, gpt_iclr,1)
ai_neur_sim_modified_para = similarity_finder(ai_neur_modified_para, gpt_neur,1)
human_iclr_sim_modified_para = similarity_finder(human_iclr_modified_para, gpt_iclr,0)
human_neur_sim_modified_para = similarity_finder(human_neur_modified_para, gpt_neur,0)

ai_iclr_sim_modified = similarity_finder(ai_iclr_modified, gpt_iclr,1)
ai_neur_sim_modified = similarity_finder(ai_neur_modified, gpt_neur,1)
human_iclr_sim_modified = similarity_finder(human_iclr_modified, gpt_iclr,0)
human_neur_sim_modified = similarity_finder(human_neur_modified, gpt_neur,0)

ai_iclr_sim_modified_para_test = similarity_finder(ai_iclr_modified_para_test, gpt_iclr,1)
ai_neur_sim_modified_para_test = similarity_finder(ai_neur_modified_para_test, gpt_neur,1)
human_iclr_sim_modified_para_test = similarity_finder(human_iclr_modified_para_test, gpt_iclr,0)
human_neur_sim_modified_para_test = similarity_finder(human_neur_modified_para_test, gpt_neur,0)

In [ ]:
ai_train = [[a[0][0]] for a in ai_iclr_sim[:608]] + [[a[0][0]] for a in ai_neur_sim[:576]] + [[a[0][0]] for a in ai_iclr_sim_modified_para] + [[a[0][0]] for a in ai_neur_sim_modified_para] + [[a[0][0]] for a in ai_iclr_sim_modified[:608]] + [[a[0][0]] for a in ai_neur_sim_modified[:576]]
human_train = [[a[0][0]] for revw in human_iclr_sim[:608]] + [[a[0][0]] for revw in human_neur_sim[:576]] + [[a[0][0]] for revw in human_iclr_sim_modified_para for a in revw] + [[a[0][0]] for revw in human_neur_sim_modified_para for a in revw] + [[a[0][0]] for a in human_iclr_sim_modified[:608]] + [[a[0][0]] for a in human_neur_sim_modified[:576]]

X_train = torch.tensor(ai_train + human_train,dtype=torch.float32)
y_train = torch.tensor([1 for i in ai_train] + [0 for i in human_train])

# Shuffle the train dataset
perm_indices = torch.randperm(len(X_train))
X_train = X_train[perm_indices]
y_train = y_train[perm_indices]

# Training the model
model = trainer(X_train,y_train)

<ipython-input-71-4daf88d32d4d>:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  X_train = torch.tensor(ai_train + human_train,dtype=torch.float32)
<ipython-input-71-4daf88d32d4d>:4: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  X_train = torch.tensor(ai_train + human_train,dtype=torch.float32)


### Testing the Modified Paraphrased Dataset:-

In [ ]:
# Testing the paraphrased model all alone
ai_test_iclr =  [[a[0][0]] for a in ai_iclr_sim_modified_para_test]
human_test_iclr =  [[a[0][0]] for revw in human_iclr_sim_modified_para_test for a in revw]
X_test_iclr = torch.tensor(ai_test_iclr+ human_test_iclr, dtype=torch.float32)
y_test_iclr = torch.tensor([1 for i in range(len(ai_test_iclr))] + [0 for i in range(len(human_test_iclr))])

# Shuffle the test dataset
perm_indices = torch.randperm(len(X_test_iclr))
X_test_iclr = X_test_iclr[perm_indices]
y_test_iclr = y_test_iclr[perm_indices]

ai_test_neur =  [[a[0][0]] for a in ai_neur_sim_modified_para_test]
human_test_neur =  [[a[0][0]] for revw in human_neur_sim_modified_para_test for a in revw]
X_test_neur = torch.tensor(ai_test_neur+ human_test_neur, dtype=torch.float32)
y_test_neur = torch.tensor([1 for i in range(len(ai_test_neur))] + [0 for i in range(len(human_test_neur))])

# Shuffle the test dataset
perm_indices = torch.randperm(len(X_test_neur))
X_test_neur = X_test_neur[perm_indices]
y_test_neur = y_test_neur[perm_indices]

print("++++++++++++++++++ ICLR ++++++++++++++++++")
tester(X_test_iclr,y_test_iclr,model)
print("\n++++++++++++++++++ NeurIPS ++++++++++++++++++")
tester(X_test_neur,y_test_neur,model)

### Testing the Modified Actual Review Dataset :-

In [ ]:
# Testing by only on the modified actual review
ai_test_iclr = [[a[0][0]] for a in ai_iclr_sim_modified[608:]]
human_test_iclr = [[a[0][0]] for a in human_iclr_sim_modified[608:]]
X_test_iclr = torch.tensor(ai_test_iclr+ human_test_iclr, dtype=torch.float32)
y_test_iclr = torch.tensor([1 for i in range(len(ai_test_iclr))] + [0 for i in range(len(human_test_iclr))])

# Shuffle the test dataset
perm_indices = torch.randperm(len(X_test_iclr))
X_test_iclr = X_test_iclr[perm_indices]
y_test_iclr = y_test_iclr[perm_indices]

ai_test_neur =  [[a[0][0]] for a in ai_neur_sim_modified[576:]]
human_test_neur = [[a[0][0]] for a in human_neur_sim_modified[576:]]
X_test_neur = torch.tensor(ai_test_neur+ human_test_neur, dtype=torch.float32)
y_test_neur = torch.tensor([1 for i in range(len(ai_test_neur))] + [0 for i in range(len(human_test_neur))])

# Shuffle the test dataset
perm_indices = torch.randperm(len(X_test_neur))
X_test_neur = X_test_neur[perm_indices]
y_test_neur = y_test_neur[perm_indices]

print("++++++++++++++++++ ICLR ++++++++++++++++++")
tester(X_test_iclr,y_test_iclr,model)
print("\n++++++++++++++++++ NeurIPS ++++++++++++++++++")
tester(X_test_neur,y_test_neur,model)

### Testing by combining all of them :-

In [ ]:
# Testing by combining them all
ai_test_iclr = [[a[0][0]] for a in ai_iclr_sim[608:]] + [[a[0][0]] for a in ai_iclr_sim_modified_para_test]  + [[a[0][0]] for a in ai_iclr_sim_modified[608:]]
human_test_iclr = [[a[0][0]] for revw in human_iclr_sim[608:]]  + [[a[0][0]] for revw in human_iclr_sim_modified_para_test for a in revw] + [[a[0][0]] for a in human_iclr_sim_modified[608:]]
X_test_iclr = torch.tensor(ai_test_iclr+ human_test_iclr, dtype=torch.float32)
y_test_iclr = torch.tensor([1 for i in range(len(ai_test_iclr))] + [0 for i in range(len(human_test_iclr))])

# Shuffle the test dataset
perm_indices = torch.randperm(len(X_test_iclr))
X_test_iclr = X_test_iclr[perm_indices]
y_test_iclr = y_test_iclr[perm_indices]

ai_test_neur = [[a[0][0]] for a in ai_neur_sim[576:]] + [[a[0][0]] for a in ai_neur_sim_modified_para_test]  + [[a[0][0]] for a in ai_neur_sim_modified[576:]]
human_test_neur = [[a[0][0]] for revw in human_neur_sim[576:]]  + [[a[0][0]] for revw in human_neur_sim_modified_para_test for a in revw] + [[a[0][0]] for a in human_neur_sim_modified[576:]]
X_test_neur = torch.tensor(ai_test_neur+ human_test_neur, dtype=torch.float32)
y_test_neur = torch.tensor([1 for i in range(len(ai_test_neur))] + [0 for i in range(len(human_test_neur))])

# Shuffle the test dataset
perm_indices = torch.randperm(len(X_test_neur))
X_test_neur = X_test_neur[perm_indices]
y_test_neur = y_test_neur[perm_indices]

print("++++++++++++++++++ ICLR ++++++++++++++++++")
tester(X_test_iclr,y_test_iclr,model)
print("\n++++++++++++++++++ NeurIPS ++++++++++++++++++")
tester(X_test_neur,y_test_neur,model)